In [2]:
import sys
sys.path.append("..")

In [3]:
from mmcv import Config, DictAction
from mmcv.runner import load_checkpoint
from models import build_posenet
import torch

/home/hoang/miniconda3/envs/ikemen/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/hoang/miniconda3/envs/ikemen/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
/home/hoang/miniconda3/envs/ikemen/lib/python3.10/site-packages/mmpose/models/registry.py:9: DeprecationWarning: Registries (BACKBONES, NECKS, HEADS, LOSSES, POSENETS) have been moved to mmpose.models.builder. Importi

In [4]:
cfg_file = "../configs/top_down/lite_hrnet/mpii/litehrnet_30_mpii_256x256.py"
cfg = Config.fromfile(cfg_file)
model = build_posenet(cfg.model)

/home/hoang/miniconda3/envs/ikemen/lib/python3.10/site-packages/mmpose/models/detectors/top_down.py:61: DeprecationWarning: `loss_pose` for TopDown is deprecated, use `loss_keypoint` for heads instead. See https://github.com/open-mmlab/mmpose/pull/382 for more information.
  warnings.warn(


In [5]:
# In một số trọng số trước khi load checkpoint
print("===> BEFORE LOADING CHECKPOINT:")
for name, param in model.named_parameters():
    if 'conv' in name:  # chỉ lọc các lớp conv cho gọn
        print(f"{name}: mean={param.data.mean().item():.6f}, std={param.data.std().item():.6f}")
    break  # chỉ in 1 layer đầu tiên để ngắn gọn (bạn có thể bỏ break để in hết)

# Load checkpoint
checkpoint = load_checkpoint(model, '../ckpts/litehrnet_30_mpii_256x256.pth', map_location='cpu')

# In lại sau khi load checkpoint
print("\n===> AFTER LOADING CHECKPOINT:")
for name, param in model.named_parameters():
    if 'conv' in name:
        print(f"{name}: mean={param.data.mean().item():.6f}, std={param.data.std().item():.6f}")
    break  # in đúng layer lúc nãy

===> BEFORE LOADING CHECKPOINT:
backbone.stem.conv1.conv.weight: mean=-0.000034, std=0.001009
load checkpoint from local path: ../ckpts/litehrnet_30_mpii_256x256.pth
The model and loaded state dict do not match exactly

size mismatch for keypoint_head.final_layer.weight: copying a param with shape torch.Size([16, 40, 1, 1]) from checkpoint, the shape in current model is torch.Size([30, 40, 1, 1]).
size mismatch for keypoint_head.final_layer.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([30]).

===> AFTER LOADING CHECKPOINT:
backbone.stem.conv1.conv.weight: mean=-0.006808, std=0.176222


In [22]:
dummy_input = torch.randn(2, 3, 256, 256)

In [23]:
backbone = model.backbone
head = model.keypoint_head

In [25]:
latent = backbone(dummy_input)
print(len(latent))

1


In [26]:
out = head(latent)
out.shape

torch.Size([2, 30, 64, 64])

In [27]:
from mmpose.core.evaluation.top_down_eval import _get_max_preds

In [28]:
preds, maxvals = _get_max_preds(out.detach().numpy())
print(preds.shape, maxvals.shape)

(2, 30, 2) (2, 30, 1)
